# **Многослойоный перцептрон** <br>


Для работы кода необходимо предоставить доступ к Google Disk, создать в корне папки Autocorrelation и Lables, загрузить в них набор данных и метки, а также создать в корне папку multilayer_perceptron для сохранения результатов обучения

Загрузка данных с Google Disk

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!ls "/content/drive/My Drive/"

 2_5305779684380783213.docx   IMG_20180624_012413.jpg
 baza.pdf		      IMG_20180624_012420.jpg
'Colab Notebooks'	      IMG_20180624_012444.jpg
 DSCF3993.jpg		      IMG_20180624_012451.jpg
 DSCF4073.jpg		      IMG_20180624_012500.jpg
 DSCF4121.jpg		      IMG_20180624_012501.jpg
 DSCF4264.jpg		      IMG_20180624_013405.jpg
 DSCF4275.jpg		      IMG_20191218_123350.jpg
 DSCF4971.jpg		      IMG_20191218_123418.jpg
 IMG_0689.jpg		      IMG_20191218_124720.jpg
 IMG_0823.jpg		      IMG_20191218_130138.jpg
 IMG_0878.jpg		      Lektsia_1_2022.ppt
 IMG_0954.jpg		      Lektsia_2_2022.ppt
 IMG_20180623_215438.jpg      Lektsia_3_2022.ppt
 IMG_20180623_215449.jpg      otvety_1.docx
 IMG_20180623_220917.jpg      otvety_2.docx
 IMG_20180623_224551.jpg      otvety_3.docx
 IMG_20180623_224604.jpg      Transmetropolitan
 IMG_20180623_224606.jpg      ULA_6552.gdoc
 IMG_20180623_224923.jpg      ULA_6675.gdoc
 IMG_20180623_225647.jpg      ULA_6679.gdoc
 IMG_20180623_225943.jpg      ULA_6681.gdoc
 IMG_2018062

Загрузка библиотеки для построения сети Кохонена

In [4]:
!pip install minisom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.1-py3-none-any.whl size=10589 sha256=4bb39e09c1ff508a23e49dec3d44ae2fd35f9560c2e973e400bd7a33769b610a
  Stored in directory: /root/.cache/pip/wheels/c7/92/d2/33bbda5f86fd8830510b16aa98c8dd420129b5cb24248fd6db
Successfully built minisom


Подключение библиотек

In [5]:
import matplotlib
matplotlib.use("Agg")
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.models import load_model
from keras.layers.core import Dense
from keras.optimizers import Adagrad
from keras.optimizers import Adadelta
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
from minisom import MiniSom
import pathlib
import pandas
import random
import pickle
import cv2
import os

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

Преобразование набора данных в формат, удобный для обучения. Эта часть кода может быть выполнена один раз, поскольку в нем преобразованные данные сохраняются на Google Disk, после чего они могут быть загружены в уже готовом виде

In [6]:
au_p = pathlib.Path("/content/drive/My Drive/Colab Notebooks/Autocorrelation/").glob('**/*')
AutocorPaths = [x for x in au_p if x.is_file()]
random.seed(40)
la_p = pathlib.Path("/content/drive/My Drive/Colab Notebooks/Labels/").glob('**/*')
LabelsPaths = [x for x in la_p if x.is_file()]
print(len(AutocorPaths))
Paths = list(zip(AutocorPaths, LabelsPaths))
random.shuffle(Paths)
#AutocorPaths, LabelsPaths = zip(*temp)
#AutocorPaths, LabelsPaths = list(AutocorPaths), list(LabelsPaths)

data = []
labels = []


for autocorpath, labelspath in Paths:
  auto_file = open(autocorpath,'r')
  label_file = open(labelspath,'r') 

  for auto_line in auto_file:
    auto_temp = auto_line.split()
    autocor_sample = [float(item) for item in auto_temp]

    label_line = label_file.readline()
    label_temp = label_line.split()
    label_sample = [int(item) for item in label_temp]

    # Функция автокорреляции записывается столько раз, сколько меток ей сопоставлено
    for label in label_sample:
      one_hot_label = [0, 0, 0, 0, 0]
      one_hot_label[label-1] = 1
      labels.append(one_hot_label)
      data.append(autocor_sample)
  

  label_file.close()
  auto_file.close()
    

print(len(labels))

data = np.array(data)
labels = np.array(labels)

with open("/content/drive/My Drive/Colab Notebooks/Kohonen_network/data.pickle", 'wb') as f:
  pickle.dump(data, f)
print("Data saved")

with open("/content/drive/My Drive/Colab Notebooks/Kohonen_network/labels.pickle", 'wb') as f:
  pickle.dump(labels, f)
print("Labels saved")

1
745
Data saved
Labels saved


Преобразование набора данных в обучающую и тестовую выборки

In [7]:
with open("/content/drive/My Drive/Colab Notebooks/Kohonen_network/data.pickle", 'rb') as f:
  data = pickle.load(f)
print("Data loaded")

with open("/content/drive/My Drive/Colab Notebooks/Kohonen_network/labels.pickle", 'rb') as f:
  labels = pickle.load(f)
print("Labels loaded")

(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.15,
                                                  random_state=40)

#(trainX, testX, train_labels_nums, test_labels_nums) = train_test_split(data, labels_nums,
#                                                  test_size=0.15,
#                                                  random_state=42)

print("Dataset prepared")

Data loaded
Labels loaded
Dataset prepared


Преобразование one hot представления меток в целые числа от 1 до 5

---



In [8]:
train_labels_nums = [];
test_labels_nums = [];

for one_hot_label in trainY:
  res = [idx for idx, val in enumerate(one_hot_label) if val != 0]
  label = res[0]+1
  train_labels_nums.append(label)

for one_hot_label in testY:
  res = [idx for idx, val in enumerate(one_hot_label) if val != 0]
  label = res[0]+1
  test_labels_nums.append(label)

Обучение сети и результаты обучения нейронной сети. Выводятся результаты применения сети к тестовой выборке и отчет о качестве работы сети. В отчете представлены значения метрик precision, recall и f1-score для каждого из пяти типовых аварийных состояний судна, а также их средние значения для всех типов вместе

In [9]:
som_pure_data = MiniSom(x= 1, y = 5, input_len = 30, neighborhood_function = 'gaussian',sigma=0.01, learning_rate=0.1)

som_pure_data.random_weights_init(trainX)
starting_weights = som_pure_data.get_weights().copy()
som_pure_data.train(trainX, 1000000)

qnt = som_pure_data.quantization(testX)
som_pure_data.quantization_error(testX)

winmap_pure_data = som_pure_data.labels_map(trainX, train_labels_nums)
default_class = np.sum(list(winmap_pure_data.values())).most_common()[0][0]
result = []
for d in trainX:
    win_position = som_pure_data.winner(d)
    if win_position in winmap_pure_data:
        result.append(winmap_pure_data[win_position].most_common()[0][0])
    else:
        result.append(default_class)
print(classification_report(train_labels_nums, result))

              precision    recall  f1-score   support

           1       0.67      0.45      0.54       128
           2       0.56      0.99      0.71       122
           3       0.58      0.43      0.50       122
           4       0.53      0.43      0.48       130
           5       0.55      0.56      0.56       131

    accuracy                           0.57       633
   macro avg       0.58      0.57      0.56       633
weighted avg       0.58      0.57      0.55       633

